In [1]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

from PIL import Image

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

# if __name__ == "__main__":
print("Entry:", sys.argv[0])
print("System argument(s):", len(sys.argv))
print("Start of \"" + sys.argv[0] + "\"")

# Overlay
ol = Overlay("/home/xilinx/Wilbur/IPBitFile/proposed_model.bit")
ip_proposed_model = ol.proposed_model_0
    
Num_train = 27455
Num_test = 7172

Golden_label_train = allocate(shape=(Num_train,), dtype=np.int32)
file= open("MNIST_DATA/labels_train.mdl", "r+")
for i in range(Num_train):
    line = file.readline()
    Golden_label_train[i] = int(line)
file.close()

Golden_label_test = np.zeros(Num_test)
file= open("MNIST_DATA/labels.mdl", "r+")
for i in range(Num_test):
    line = file.readline()
    Golden_label_test[i] = int(line)
file.close()

Buf_input = allocate(shape=(1024,), dtype=np.float32)
    
Buf_Wconv1 = allocate(shape=(150,), dtype=np.float32)
file= open("Param/Wconv1.mdl", "r+")
for i in range(150):
    line = file.readline()
    Buf_Wconv1[i] = float(line)
file.close()
    
Buf_bconv1 = allocate(shape=(6,), dtype=np.float32)
file= open("Param/bconv1.mdl", "r+")
for i in range(6):
    line = file.readline()
    Buf_bconv1[i] = float(line)
file.close()
    
Buf_Wconv2 = allocate(shape=(2400,), dtype=np.float32)
file= open("Param/Wconv2.mdl", "r+")
for i in range(2400):
    line = file.readline()
    Buf_Wconv2[i] = float(line)
file.close()

Buf_bconv2 = allocate(shape=(16,), dtype=np.float32)
file= open("Param/bconv2.mdl", "r+")
for i in range(16):
    line = file.readline()
    Buf_bconv2[i] = float(line)
file.close()

Buf_Wconv3 = allocate(shape=(51200,), dtype=np.float32)
file= open("Param/Wconv3.mdl", "r+")
for i in range(51200):
    line = file.readline()
    Buf_Wconv3[i] = float(line)
file.close()

Buf_bconv3 = allocate(shape=(128,), dtype=np.float32)
file= open("Param/bconv3.mdl", "r+")
for i in range(128):
    line = file.readline()
    Buf_bconv3[i] = float(line)
file.close()

Buf_Welm = allocate(shape=(21504,), dtype=np.float32)
file= open("Param/Welm.mdl", "r+")
for i in range(21504):
    line = file.readline()
    Buf_Welm[i] = float(line)
file.close()

Buf_belm = allocate(shape=(168,), dtype=np.float32)
file= open("Param/belm.mdl", "r+")
for i in range(168):
    line = file.readline()
    Buf_belm[i] = float(line)
file.close()

Buf_beta_elm = allocate(shape=(4368,), dtype=np.float32)
file= open("Param/beta_elm.mdl", "r+")
for i in range(4368):
    line = file.readline()
    Buf_beta_elm[i] = float(line)
file.close()

Buf_output = allocate(shape=(26,), dtype=np.float32)

print("\n====================\nModel Loading Done!\n====================")

Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"

Model Loading Done!


### ======================= Test ===========================

In [2]:
# # Execute at PL side
timeKernelStart = time()

file= open("MNIST_DATA/images.mdl", "r+")
error = 0

for i in range(Num_test):
    # print("================================ i = {} ================================".format(i))
    
    ip_proposed_model.write(0x68, 0x00)   # Golden_label = 0
    ip_proposed_model.write(0x70, 0x00)   # stage_train = 0
    ip_proposed_model.write(0x78, 0x00)   # BS = 0
    
    if(i==0):
        ip_proposed_model.write(0x80, 0x01)   # init = 1
    else: 
        ip_proposed_model.write(0x80, 0x00)   # init = 0
        
    for j in range(1024):
        line = file.readline()
        Buf_input[j] = float(line)
    
    ip_proposed_model.write(0x10, Buf_input.device_address)
    ip_proposed_model.write(0x18, Buf_Wconv1.device_address)
    ip_proposed_model.write(0x20, Buf_bconv1.device_address)   
    ip_proposed_model.write(0x28, Buf_Wconv2.device_address)
    ip_proposed_model.write(0x30, Buf_bconv2.device_address)
    ip_proposed_model.write(0x38, Buf_Wconv3.device_address)
    ip_proposed_model.write(0x40, Buf_bconv3.device_address)
    ip_proposed_model.write(0x48, Buf_Welm.device_address)
    ip_proposed_model.write(0x50, Buf_belm.device_address)
    ip_proposed_model.write(0x58, Buf_beta_elm.device_address)
    ip_proposed_model.write(0x60, Buf_output.device_address)
    
    ip_proposed_model.write(0x00, 0x01)
    while (ip_proposed_model.read(0x00) & 0x4) == 0x0:
        continue
        
    # print("Buf_output:",Buf_output)
    Predict = np.argmax(Buf_output)
    # print("Predict: {},  Golden: {}".format(Predict,int(Golden_label[i])))
    if(Predict!=int(Golden_label_test[i])):
        error += 1
    
timeKernelEnd = time()

print("============================")

correct = Num_test-error
print("accuracy : {}/{} ({:2.2f}%) ".format(correct,Num_test,(correct/Num_test)*100))

print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s") 

file.close()

accuracy : 6365/7172 (88.75%) 
Kernel execution time: 182.7164466381073 s


### ======================= Train (QRD mode) ===========================
There are still some bugs in this part...

In [3]:
# # Execute at PL side
timeKernelStart = time()

file= open("MNIST_DATA/images_train.mdl", "r+")
error = 0

print("========== QRD start ==========")
for i in range(Num_train):
    if(i%100==0):
        print("================================ i = {} ================================".format(i))
        
    ip_proposed_model.write(0x68, int(Golden_label_train[i]))   # Golden_label = Golden_label_train[i]
    ip_proposed_model.write(0x70, 0x01)   # stage_train = 1
    ip_proposed_model.write(0x78, 0x00)   # BS = 0
    
    if(i==0):
        ip_proposed_model.write(0x80, 0x01)   # init = 1
    else: 
        ip_proposed_model.write(0x80, 0x00)   # init = 0
        
    for j in range(1024):
        line = file.readline()
        Buf_input[j] = float(line)
    
    ip_proposed_model.write(0x10, Buf_input.device_address)
    ip_proposed_model.write(0x18, Buf_Wconv1.device_address)
    ip_proposed_model.write(0x20, Buf_bconv1.device_address)   
    ip_proposed_model.write(0x28, Buf_Wconv2.device_address)
    ip_proposed_model.write(0x30, Buf_bconv2.device_address)
    ip_proposed_model.write(0x38, Buf_Wconv3.device_address)
    ip_proposed_model.write(0x40, Buf_bconv3.device_address)
    ip_proposed_model.write(0x48, Buf_Welm.device_address)
    ip_proposed_model.write(0x50, Buf_belm.device_address)
    ip_proposed_model.write(0x58, Buf_beta_elm.device_address)
    ip_proposed_model.write(0x60, Buf_output.device_address)
    
    ip_proposed_model.write(0x00, 0x01)
    while (ip_proposed_model.read(0x00) & 0x4) == 0x0:
        continue
    
timeKernelEnd = time()

print("========== QRD completed ==========\n")
print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s") 

file.close()

========== QRD start ==========
================================ i = 0 ================================


KeyboardInterrupt: 

### ======================= Test (BS mode) ===========================

In [4]:
print("========== BS start ==========")

timeKernelStart = time()

ip_proposed_model.write(0x68, int(Golden_label_train[0]))   # Golden_label = 0
ip_proposed_model.write(0x70, 0x01)   # stage_train = 1
ip_proposed_model.write(0x78, 0x01)   # BS = 1

ip_proposed_model.write(0x80, 0x00)   # init = 0

ip_proposed_model.write(0x10, Buf_input.device_address)
ip_proposed_model.write(0x18, Buf_Wconv1.device_address)
ip_proposed_model.write(0x20, Buf_bconv1.device_address)   
ip_proposed_model.write(0x28, Buf_Wconv2.device_address)
ip_proposed_model.write(0x30, Buf_bconv2.device_address)
ip_proposed_model.write(0x38, Buf_Wconv3.device_address)
ip_proposed_model.write(0x40, Buf_bconv3.device_address)
ip_proposed_model.write(0x48, Buf_Welm.device_address)
ip_proposed_model.write(0x50, Buf_belm.device_address)
ip_proposed_model.write(0x58, Buf_beta_elm.device_address)
ip_proposed_model.write(0x60, Buf_output.device_address)

ip_proposed_model.write(0x00, 0x01)
while (ip_proposed_model.read(0x00) & 0x4) == 0x0:
    continue
    
timeKernelEnd = time()

print("========== BS completed ==========")

print("============================")
print("Re-train complete !")

print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s") 

========== BS start ==========
========== BS completed ==========
Re-train complete !
Kernel execution time: 3.047008991241455 s


### ======================= Re-Test ===========================

In [5]:
# # Execute at PL side
timeKernelStart = time()

file= open("MNIST_DATA/images.mdl", "r+")
error = 0

for i in range(Num_test):
    # print("================================ i = {} ================================".format(i))
    
    ip_proposed_model.write(0x68, 0x00)   # Golden_label = 0
    ip_proposed_model.write(0x70, 0x00)   # stage_train = 0
    ip_proposed_model.write(0x78, 0x00)   # BS = 0
    
    
    if(i==0):
        ip_proposed_model.write(0x80, 0x01)   # init = 1
    else: 
        ip_proposed_model.write(0x80, 0x00)   # init = 0
        
    for j in range(1024):
        line = file.readline()
        Buf_input[j] = float(line)
    
    ip_proposed_model.write(0x10, Buf_input.device_address)
    ip_proposed_model.write(0x18, Buf_Wconv1.device_address)
    ip_proposed_model.write(0x20, Buf_bconv1.device_address)   
    ip_proposed_model.write(0x28, Buf_Wconv2.device_address)
    ip_proposed_model.write(0x30, Buf_bconv2.device_address)
    ip_proposed_model.write(0x38, Buf_Wconv3.device_address)
    ip_proposed_model.write(0x40, Buf_bconv3.device_address)
    ip_proposed_model.write(0x48, Buf_Welm.device_address)
    ip_proposed_model.write(0x50, Buf_belm.device_address)
    ip_proposed_model.write(0x58, Buf_beta_elm.device_address)
    ip_proposed_model.write(0x60, Buf_output.device_address)
    
    ip_proposed_model.write(0x00, 0x01)
    while (ip_proposed_model.read(0x00) & 0x4) == 0x0:
        continue
        
    # print("Buf_output:",Buf_output)
    Predict = np.argmax(Buf_output)
    # print("Predict: {},  Golden: {}".format(Predict,int(Golden_label_test[i])))
    if(Predict!=int(Golden_label_test[i])):
        error += 1
    
timeKernelEnd = time()

print("============================")

correct = Num_test-error
print("accuracy : {}/{} ({:2.2f}%) ".format(correct,Num_test,(correct/Num_test)*100))

print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s") 

file.close()

accuracy : 593/7172 (8.27%) 
Kernel execution time: 182.46354699134827 s
